In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("/tf/ProjectGabriel/pilca")

import numpy as np
import pandas as pd
from lightcurve_fitting import models, filters, lightcurve, fitting
import matplotlib.pyplot as plt
import matplotlib as mpl
import importlib
from utils.utils import load_lc
from utils.utils import light_curve_plot
import torch
import utils.torchphysics as tp
import utils.utils as ut
# import lc
import os
torch.set_default_dtype(torch.float64)



In [2]:
all_filters = ["z", "y", "i", "r", "g", "u", "uvw1"][::-1][-2:]  # from red to UV
mjd_array = np.linspace(3, 13, 600)


filter_combinations = [all_filters[:i+1] for i in range(len(all_filters))]

max_days = 5
time_spans = np.arange(1, max_days + 1)  # [1, 2, ..., 10]

model_parameters = [1.2, 2., 4.0, 2.5]

# --- setup light curve builder ---
builder = ut.LCBuilder(
    model_name="sc4",
    model_parameters=model_parameters,
    model_units=[1,1,1,1],
    seed=42
)

lc = builder.build_sim_lc(
    mjd_array=mjd_array,
    filters_list=all_filters,  # full set
    redshift=0.00526,
    dlum_factor=1e-1/2,
    dm=31.1,
    dL=19.,
    dLerr=2.9
)

In [3]:
lc

MJD,lum,dlum,loglum,dloglum,filter
float64,float64,float64,float64,float64,object
3.0,3.0913625485228896e+19,3.3093253501158436e+18,19.49014994119714,0.04263189912973366,y
3.01669449081803,3.0666040663493235e+19,1.4731057991492646e+18,19.486657707218324,0.0065828464366446925,z
3.03338898163606,4.923311239883422e+19,6.132590756976553e+18,19.69225729169217,0.026212123530003746,y
3.05008347245409,3.782671214880942e+19,4.366278085235634e+18,19.57779859459216,0.022664298568928404,y
3.0667779632721204,1.7185697311833668e+19,2.1456801115057702e+18,19.235167158384073,0.016376604033303845,y
3.0834724540901504,4.068833014511087e+19,1.3598517431111459e+19,19.609469866711358,0.016125658106294814,z
3.1001669449081803,2.958032766606035e+19,1.7765902763448734e+19,19.47100298043754,0.0029283568229772814,y
3.1168614357262103,4.246320687903605e+19,4.001288018235311e+18,19.628012789577454,0.0913100622848045,y
3.1335559265442403,2.9960451942106616e+19,9.475349163813714e+18,19.47654836024514,0.003548978980249753,y


In [4]:
ut.storage_parent_path

'/tf/astrodados2/pilca.storage/'

In [ ]:
t_span = 10
min_mjd = lc['MJD'].min()

lc_early = lc.where(MJD_min=min_mjd-1, MJD_max=min_mjd+t_span) 

model = models.ModifiedShockCooling4(lc_early)


ten13cmtoRsol = 1e13*1.4374e-11 
ten8p5cmstoten3kms = 10**.5
# units_array = np.array([ten8p5cmstoten3kms, 1, 1, ten13cmtoRsol, 1, 1]).reshape(1,-1)

priors = [
models.UniformPrior(0, 10),
models.UniformPrior(0, 10),
models.UniformPrior(0, 100),
models.UniformPrior(min_mjd-2, min_mjd-1+3),
models.LogUniformPrior(0,1e2),
]

p_lo = [0.5, 0.5,  1, 1.4, 6]
p_up = [3, 4,  6, 4, 8]

N_WALKERS = 32
N_STEPS = 4000
N_STEPS_BURNIN = 1 
sampler = fitting.lightcurve_mcmc(lc_early, model, 
                                priors=priors, p_lo=p_lo, p_up=p_up,
                                nwalkers=N_WALKERS, nsteps=N_STEPS, nsteps_burnin=N_STEPS_BURNIN, use_sigma=True)


save_dir = os.path.join(ut.storage_parent_path, "experiments", "mcmc")
os.mkdir(save_dir)
np.save(os.path.join(save_dir, "flatchain_mcmc_10d_all_filters.npy"), np.array(sampler.flatchain))


 Burn-in:   0%|          | 0/1 [00:00<?, ?it/s]

Sampling:  10%|▉         | 399/4000 [01:33<15:10,  3.95it/s]

In [ ]:
min_mjd

3.0